In [3]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [4]:
# 코드 8-24 VAE 인코더 네트워크
import keras
from keras import layers
from keras import backend as K
from keras.models import Model
import numpy as np

img_shape = (28, 28, 1)
batch_size = 16
latent_dim = 2 # 잠재 공간의 차원 : 2D 평면

input_img = keras.Input(shape = img_shape)

x = layers.Conv2D(32, 3,
                 padding="same", activation="relu")(input_img)
x = layers.Conv2D(64, 3,
                 padding="same", activation="relu",
                 strides=(2, 2))(x)
x = layers.Conv2D(64, 3,
                 padding="same", activation="relu")(x)
x = layers.Conv2D(64, 3,
                 padding="same", activation="relu")(x)
shape_before_flattening = K.int_shape(x)

x = layers.Flatten()(x)
x = layers.Dense(32, activation="relu")(x)

# 입력 이미지는 결국 2개의 파라미터로 인코딩됩니다
z_mean = layers.Dense(latent_dim)(x)
z_log_var = layers.Dense(latent_dim)(x)

In [5]:
# 코드 8-24 잠재 공간 샘플링 함수
def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape = (K.shape(z_mean)[0], latent_dim),
                             mean = 0., stddev=1.)
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

z = layers.Lambda(sampling)([z_mean, z_log_var])

In [6]:
# 코드 8-25 잠재 공간 포인트를 이미지로 매핑하는 VAE 디코더 네트워크
decoder_input = layers.Input(K.int_shape(z)[1:]) # input에 z를 주입합니다

# 입력을 업샘플링합니다
x = layers.Dense(np.prod(shape_before_flattening[1:]),
                activation="relu")(decoder_input)

# 인코더 모델의 마지막 Flatten 층 직전의 특성 맵과 같은 크기를 가진 특성 맵으로 z의 크기를 바꿉니다
x = layers.Reshape(shape_before_flattening[1:])(x)

# z를 원본 입력 이미지와 같은 크기의 특성 맵으로 디코딩합니다
x = layers.Convolution2DTranspose(32, 3,
                                 padding = "same",
                                 activation="relu",
                                 strides=(2, 2))(x)
x = layers.Conv2D(1, 3, padding="same",
                 activation="sigmoid")(x)

decoder = Model(decoder_input, x) # decoder_input을 디코딩된 이미지로 변환하는 디코더 모델의 객체를 만듭니다

z_decoded = decoder(z) # 모델에 z를 투입하면 디코딩된 z를 출력합니다

In [7]:
# 코드 8-26 VAE 손실을 계산하기 위해 직접 만든 층
class CustomVariationalLayer(keras.layers.Layer):
    
    def vae_loss(self, x, z_decoded):
        x = K.flatten(x)
        z_decoded = K.flatten(z_decoded)
        xent_loss = keras.metrics.binary_crossentropy(x, z_decoded)
        kl_loss = -5e-4 * K.mean(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis = -1)
        return K.mean(xent_loss + kl_loss)
    
    def call(self, inputs): # call 메서드가 있는 층을 구현합니다
        x = inputs[0]
        z_decoded = inputs[1]
        loss = self.vae_loss(x, z_decoded)
        self.add_loss(loss, inputs = inputs)
        return x # 이 출력을 사용하진 않지만 층은 무언가를 반환해야 합니다
    
y = CustomVariationalLayer()([input_img, z_decoded]) # 입력과 디코딩된 출력으로 이 층을 호출하여 모델의 최종 출력을 얻습니다

In [8]:
# 코드 8-27 VAE 훈련하기
from keras.datasets import mnist

vae = Model(input_img, y)
vae.compile(optimizer = "rmsprop",
           loss = None)

In [9]:
vae.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 28, 28, 32)   320         input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_5 (Conv2D)               (None, 14, 14, 64)   18496       conv2d_4[0][0]                   
__________________________________________________________________________________________________
conv2d_6 (Conv2D)               (None, 14, 14, 64)   36928       conv2d_5[0][0]                   
____________________________________________________________________________________________

In [10]:
(x_train, _), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype("float32") / 255
x_train = x_train.reshape(x_train.shape + (1,))
x_test = x_test.astype("float32") / 255
x_test = x_test.reshape(x_test.shape + (1,))

In [11]:
vae.fit(x=x_train, y=None,
       shuffle=True,
       epochs=10,
       batch_size=batch_size,
       validation_data=(x_test,None))

Epoch 1/10


_SymbolicException: Inputs to eager execution function cannot be Keras symbolic tensors, but found [<tf.Tensor 'dense_5/Identity:0' shape=(None, 2) dtype=float32>, <tf.Tensor 'dense_4/Identity:0' shape=(None, 2) dtype=float32>]